In [1]:
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
import nltk
nltk.download('punkt')

W0710 06:31:00.955379 140553886504832 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from nltk.tokenize import word_tokenize
import re
import collections
import pickle
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#default path for the folder inside google drive
default_path = "drive/My Drive/Colab Notebooks/"

#path for getting training data
train_article_path = default_path + "sumdata/train/train.article.txt" 

#path for training text output (headline)
train_title_path   = default_path + "sumdata/train/train.title.txt"

#path for validation text (article)
valid_article_path = default_path + "sumdata/train/valid.article.filter.txt"

#path for validation text output(headline)
valid_title_path   = default_path + "sumdata/train/valid.title.filter.txt"
input_size_ = 200000

In [0]:
def clean_str(sentence):
    sentence = re.sub("[#]+", "#", sentence)
    return sentence

In [0]:
def get_text_list(data_path, toy):
    with open (data_path, "r", encoding="utf-8") as f:
        if not toy:
            return [clean_str(x.strip()) for x in f.readlines()][:input_size_]
        else:
            return [clean_str(x.strip()) for x in f.readlines()][:1000]

In [0]:
def build_dict(step, toy=False):
    if step == "train":
        train_article_list = get_text_list(train_article_path, toy)
        train_title_list = get_text_list(train_title_path, toy)

        '''words = list()
        for sentence in train_article_list + train_title_list:
            for word in word_tokenize(sentence):
                words.append(word)

        word_counter = collections.Counter(words).most_common()
        #print(word_counter)
        # ('#', 363119), ('the', 307009), (',', 204614), ('to', 202066) ouuput 
        word_dict = dict()
        word_dict["<padding>"] = 0
        word_dict["<unk>"] = 1
        word_dict["<s>"] = 2
        word_dict["</s>"] = 3
        # we are creating the word to int dictionary 
        for word, _ in word_counter:
            word_dict[word] = len(word_dict) 

        with open(default_path + "word_dict.pickle", "wb") as f:
            pickle.dump(word_dict, f)'''
        with open(default_path + "word_dict.pickle", "rb") as f:
            word_dict = pickle.load(f)

    elif step == "valid":
        with open(default_path + "word_dict.pickle", "rb") as f:
            word_dict = pickle.load(f)
     

    reversed_dict = dict(zip(word_dict.values(), word_dict.keys()))

    '''article_max_len = max([len(word_tokenize(d)) for d in train_article_list])
    print(article_max_len)
    summary_max_len = max([len(word_tokenize(d)) for d in train_title_list])
    print(summary_max_len)'''
    article_max_len = 150
    summary_max_len = 50

    return word_dict, reversed_dict, article_max_len, summary_max_len

In [0]:
def build_dataset(step,article_list, title_list, word_dict, article_max_len, summary_max_len, toy=False):
    if step == "train":
        article_list = [clean_str(x.strip()) for x in article_list]
        title_list   = [clean_str(x.strip()) for x in title_list]
    elif step == "valid":
        article_list = [clean_str(x.strip()) for x in article_list]
    else:
        raise NotImplementedError
    #print("Type of batch ",type(article_list))
    #print("TYpe of batch ",type(title_list))
    #print(article_list)
    #print("Next")
    #print(title_list)
    x = [word_tokenize(d) for d in article_list]
    x = [[word_dict.get(w, word_dict["<unk>"]) for w in d] for d in x]
    #print("Hi Prabha, here you are")
    #print(x)
    x = [d[:article_max_len] for d in x]
    x = [d + (article_max_len - len(d)) * [word_dict["<padding>"]] for d in x]
    
    if step == "valid":
        return x
    else:        
        y = [word_tokenize(d) for d in title_list]
        y = [[word_dict.get(w, word_dict["<unk>"]) for w in d] for d in y]
        y = [d[:(summary_max_len - 1)] for d in y]
        return x, y

In [0]:
def get_init_embedding(reversed_dict, embedding_size):
    #glove_file = default_path + "glove/glove.6B.300d.txt"
    #word2vec_file = get_tmpfile(default_path + "word2vec_format.vec")
    #glove2word2vec(glove_file, word2vec_file)
    print("Loading Glove vectors...")
    #word_vectors = KeyedVectors.load_word2vec_format(word2vec_file)

    with open(default_path + "glove/model_glove_300.pkl", 'rb') as handle:
        word_vectors = pickle.load(handle)
        
    word_vec_list = list()
    for _, word in sorted(reversed_dict.items()):
        try:
            word_vec = word_vectors.word_vec(word)
        except KeyError:
            word_vec = np.zeros([embedding_size], dtype=np.float32)

        word_vec_list.append(word_vec)

    # Assign random vector to <s>, </s> token
    word_vec_list[2] = np.random.normal(0, 1, embedding_size)
    word_vec_list[3] = np.random.normal(0, 1, embedding_size)

    return np.array(word_vec_list)

In [0]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [0]:
class Model(object):
    def __init__(self, reversed_dict, article_max_len, summary_max_len, args, forward_only=False):
        self.vocabulary_size = len(reversed_dict)
        self.embedding_size = args.embedding_size
        self.num_hidden = args.num_hidden
        self.num_layers = args.num_layers
        self.learning_rate = args.learning_rate
        self.beam_width = args.beam_width
        
        if not forward_only:
            self.keep_prob = args.keep_prob
        else:
            self.keep_prob = 1.0
        self.cell = tf.nn.rnn_cell.BasicLSTMCell
        with tf.variable_scope("decoder/projection"):
            self.projection_layer = tf.layers.Dense(self.vocabulary_size, use_bias=False)

        self.batch_size = tf.placeholder(tf.int32, (), name="batch_size")
        self.X = tf.placeholder(tf.int32, [None, article_max_len])
        self.X_len = tf.placeholder(tf.int32, [None])
        self.decoder_input = tf.placeholder(tf.int32, [None, summary_max_len])
        self.decoder_len = tf.placeholder(tf.int32, [None])
        self.decoder_target = tf.placeholder(tf.int32, [None, summary_max_len])
        self.global_step = tf.Variable(0, trainable=False)
        with tf.name_scope("embedding"):
            if not forward_only and args.glove:
                init_embeddings = tf.constant(get_init_embedding(reversed_dict, self.embedding_size), dtype=tf.float32)
            else:
                init_embeddings = tf.random_uniform([self.vocabulary_size, self.embedding_size], -1.0, 1.0)
            self.embeddings = tf.get_variable("embeddings", initializer=init_embeddings)
            self.encoder_emb_inp = tf.transpose(tf.nn.embedding_lookup(self.embeddings, self.X), perm=[1, 0, 2])
            self.decoder_emb_inp = tf.transpose(tf.nn.embedding_lookup(self.embeddings, self.decoder_input), perm=[1, 0, 2])

        with tf.name_scope("encoder"):
            fw_cells = [self.cell(self.num_hidden) for _ in range(self.num_layers)]
            bw_cells = [self.cell(self.num_hidden) for _ in range(self.num_layers)]
            fw_cells = [rnn.DropoutWrapper(cell) for cell in fw_cells]
            bw_cells = [rnn.DropoutWrapper(cell) for cell in bw_cells]

            encoder_outputs, encoder_state_fw, encoder_state_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
                fw_cells, bw_cells, self.encoder_emb_inp,
                sequence_length=self.X_len, time_major=True, dtype=tf.float32)
            self.encoder_output = tf.concat(encoder_outputs, 2)
            encoder_state_c = tf.concat((encoder_state_fw[0].c, encoder_state_bw[0].c), 1)
            encoder_state_h = tf.concat((encoder_state_fw[0].h, encoder_state_bw[0].h), 1)
            self.encoder_state = rnn.LSTMStateTuple(c=encoder_state_c, h=encoder_state_h)

        with tf.name_scope("decoder"), tf.variable_scope("decoder") as decoder_scope:
            decoder_cell = self.cell(self.num_hidden * 2)

            if not forward_only:
                attention_states = tf.transpose(self.encoder_output, [1, 0, 2])
                attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                    self.num_hidden * 2, attention_states, memory_sequence_length=self.X_len, normalize=True)
                decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism,
                                                                   attention_layer_size=self.num_hidden * 2)
                initial_state = decoder_cell.zero_state(dtype=tf.float32, batch_size=self.batch_size)
                initial_state = initial_state.clone(cell_state=self.encoder_state)
                helper = tf.contrib.seq2seq.TrainingHelper(self.decoder_emb_inp, self.decoder_len, time_major=True)
                decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, initial_state)
                outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, output_time_major=True, scope=decoder_scope)
                self.decoder_output = outputs.rnn_output
                self.logits = tf.transpose(
                    self.projection_layer(self.decoder_output), perm=[1, 0, 2])
                self.logits_reshape = tf.concat(
                    [self.logits, tf.zeros([self.batch_size, summary_max_len - tf.shape(self.logits)[1], self.vocabulary_size])], axis=1)
            else:
                tiled_encoder_output = tf.contrib.seq2seq.tile_batch(
                    tf.transpose(self.encoder_output, perm=[1, 0, 2]), multiplier=self.beam_width)
                tiled_encoder_final_state = tf.contrib.seq2seq.tile_batch(self.encoder_state, multiplier=self.beam_width)
                tiled_seq_len = tf.contrib.seq2seq.tile_batch(self.X_len, multiplier=self.beam_width)
                attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                    self.num_hidden * 2, tiled_encoder_output, memory_sequence_length=tiled_seq_len, normalize=True)
                decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism,
                                                                   attention_layer_size=self.num_hidden * 2)
                initial_state = decoder_cell.zero_state(dtype=tf.float32, batch_size=self.batch_size * self.beam_width)
                initial_state = initial_state.clone(cell_state=tiled_encoder_final_state)
                decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                    cell=decoder_cell,
                    embedding=self.embeddings,
                    start_tokens=tf.fill([self.batch_size], tf.constant(2)),
                    end_token=tf.constant(3),
                    initial_state=initial_state,
                    beam_width=self.beam_width,
                    output_layer=self.projection_layer
                )
                outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
                    decoder, output_time_major=True, maximum_iterations=summary_max_len, scope=decoder_scope)
                self.prediction = tf.transpose(outputs.predicted_ids, perm=[1, 2, 0])
        #print("Decoder output ",self.decoder_output.get_shape().as_list())
        #print("Decoder input ",self.decoder_input.get_shape().as_list())
        #print("input ",self.X.get_shape().as_list())

        with tf.name_scope("loss"):
            if not forward_only:
                crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits=self.logits_reshape, labels=self.decoder_target)
                weights = tf.sequence_mask(self.decoder_len, summary_max_len, dtype=tf.float32)
                self.loss = tf.reduce_sum(crossent * weights / tf.to_float(self.batch_size))

                params = tf.trainable_variables()
                gradients = tf.gradients(self.loss, params)
                clipped_gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
                optimizer = tf.train.AdamOptimizer(self.learning_rate)
                self.update = optimizer.apply_gradients(zip(clipped_gradients, params), global_step=self.global_step)

In [12]:
'''import time
start = time.perf_counter()
import tensorflow as tf
import argparse
import pickle
import os

class args:
    pass
  
args.num_hidden=150
args.num_layers=2
args.beam_width=10
args.glove="store_true"
args.embedding_size=300

args.learning_rate=1e-3
args.batch_size = 50
args.num_epochs = 5
args.keep_prob = 0.8

args.toy = False #"store_true"

args.with_model="store_true"
if not os.path.exists(default_path + "saved_model"):
    os.mkdir(default_path + "saved_model")
else:
  old_model_checkpoint_path = open(default_path + 'saved_model/checkpoint', 'r')
  old_model_checkpoint_path = "".join([default_path + "saved_model/",old_model_checkpoint_path.read().splitlines()[0].split('"')[1] ])


print("Building dictionary...")
word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("train", args.toy)
print("Loading training dataset...")
#train_x, train_y = build_dataset("train", word_dict, article_max_len, summary_max_len, args.toy)

tf.reset_default_graph()

with tf.Session() as sess:
    model = Model(reversed_dict, article_max_len, summary_max_len, args)
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(tf.global_variables())
    if 'old_model_checkpoint_path' in globals():
        print("Continuing from previous trained model:" , old_model_checkpoint_path , "...")
        saver.restore(sess, old_model_checkpoint_path )

    #batches = batch_iter(train_x, train_y, args.batch_size, args.num_epochs)
    num_batches_per_epoch = (input_size_) // args.batch_size

    print("\nIteration starts.")
    print("Number of batches per epoch :", num_batches_per_epoch)
    for inde in range(args.num_epochs):
        with open(default_path + "sumdata/train/train.article.txt", "r", encoding="utf-8") as f1, open(default_path + "sumdata/train/train.title.txt", "r", encoding="utf-8") as f2:
            print("Hello")
            inputs_ = list()
            outputs_ = list()
            for index, (inputs, outputs) in enumerate(zip(f1, f2)):
                if((index+1) % args.batch_size == 0 ):
                    #print(inputs_)
                    inputss, outputss = build_dataset("train", inputs_, outputs_, word_dict, article_max_len, summary_max_len, args.toy)
                    #print("here it is working better")
                    inputss = np.array(inputss)
                    outputss = np.array(outputss)
                    #print(len(inputss))
                    #print(len(outputss))
                    #inputss = inputss[:,:]
                    #outputs = outputs[:,:]
                    #print("After ",len(inputss))
                    #print("After ",len(outputss))
                    batch_x_len = list(map(lambda x: len([y for y in x if y != 0]), inputss))
                    batch_decoder_input = list(map(lambda x: [word_dict["<s>"]] + list(x), outputss))
                    batch_decoder_len = list(map(lambda x: len([y for y in x if y != 0]), batch_decoder_input))
                    batch_decoder_output = list(map(lambda x: list(x) + [word_dict["</s>"]], outputss))

                    batch_decoder_input = list(
                        map(lambda d: d + (summary_max_len - len(d)) * [word_dict["<padding>"]], batch_decoder_input))
                    batch_decoder_output = list(
                        map(lambda d: d + (summary_max_len - len(d)) * [word_dict["<padding>"]], batch_decoder_output))

                    train_feed_dict = {
                        model.batch_size: len(inputss),
                        model.X: inputss,
                        model.X_len: batch_x_len,
                        model.decoder_input: batch_decoder_input,
                        model.decoder_len: batch_decoder_len,
                        model.decoder_target: batch_decoder_output
                    }

                    _, step, loss = sess.run([model.update, model.global_step, model.loss], feed_dict=train_feed_dict)
                    #print("Hey it is running Prabha")
                    if step % 1000 == 0:
                        print("step {0}: loss = {1}".format(step, loss))
                    print("step {0}: loss = {1}".format(step, loss))
                    if step % num_batches_per_epoch == 0:
                        hours, rem = divmod(time.perf_counter() - start, 3600)
                        minutes, seconds = divmod(rem, 60)
                        saver.save(sess, default_path + "saved_model/model.ckpt", global_step=step)
                        print(" Epoch {0}: Model is saved.".format(step // num_batches_per_epoch),
                        "Elapsed: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds) , "\n")
                    inputs_, outputs_ = [],[]
                    inputs_.append(inputs)
                    outputs_.append(outputs)
                else:
                    inputs_.append(inputs)
                    outputs_.append(outputs)'''

'import time\nstart = time.perf_counter()\nimport tensorflow as tf\nimport argparse\nimport pickle\nimport os\n\nclass args:\n    pass\n  \nargs.num_hidden=150\nargs.num_layers=2\nargs.beam_width=10\nargs.glove="store_true"\nargs.embedding_size=300\n\nargs.learning_rate=1e-3\nargs.batch_size = 50\nargs.num_epochs = 5\nargs.keep_prob = 0.8\n\nargs.toy = False #"store_true"\n\nargs.with_model="store_true"\nif not os.path.exists(default_path + "saved_model"):\n    os.mkdir(default_path + "saved_model")\nelse:\n  old_model_checkpoint_path = open(default_path + \'saved_model/checkpoint\', \'r\')\n  old_model_checkpoint_path = "".join([default_path + "saved_model/",old_model_checkpoint_path.read().splitlines()[0].split(\'"\')[1] ])\n\n\nprint("Building dictionary...")\nword_dict, reversed_dict, article_max_len, summary_max_len = build_dict("train", args.toy)\nprint("Loading training dataset...")\n#train_x, train_y = build_dataset("train", word_dict, article_max_len, summary_max_len, args.toy

In [0]:
def get_text_list_test(data_path, toy,input_size_):
    with open (data_path, "r", encoding="utf-8") as f:
        if not toy:
            return [clean_str(x.strip()) for x in f.readlines()][:input_size_]
        else:
            return [clean_str(x.strip()) for x in f.readlines()][:1000]

In [14]:
import tensorflow as tf
import pickle
#from model import Model
#from utils import build_dict, build_dataset, batch_iter

#path for validation text (article)
valid_article_path = default_path + "sumdata/train/valid.article.filter.txt"

#path for validation text output(headline)
valid_title_path   = default_path + "sumdata/train/valid.title.filter.txt"


#with open("args.pickle", "rb") as f:
#    args = pickle.load(f)

tf.reset_default_graph()

class args:
    pass
  
args.num_hidden=150
args.num_layers=2
args.beam_width=10
args.glove="store_true"
args.embedding_size = 300

args.learning_rate=1e-3
args.batch_size= 1
args.num_epochs= 1
args.keep_prob = 0.8

args.toy = False

args.with_model="store_true"
args.input_size_ = 1


print("Loading dictionary...")
print("Loading dictionary...")
word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("valid", args.toy)
print("Loading validation dataset...")
#valid_x = build_dataset("valid", word_dict, article_max_len, summary_max_len, args.toy)
#valid_x_len = [len([y for y in x if y != 0]) for x in valid_x]
print("Loading article and reference...")
article = get_text_list_test(valid_article_path, args.toy, args.input_size_)
reference = get_text_list_test(valid_title_path, args.toy, args.input_size_)

with tf.Session() as sess:
    print("Loading saved model...")
    model = Model(reversed_dict, article_max_len, summary_max_len, args, forward_only=True)
    saver = tf.train.Saver(tf.global_variables())
    ckpt = tf.train.get_checkpoint_state(default_path + "saved_model/")
    saver.restore(sess, ckpt.model_checkpoint_path)

    #batches = batch_iter(valid_x, [0] * len(valid_x), args.batch_size, 1)

    print("Writing summaries to 'result.txt'...")
    for inde in range(args.num_epochs):
        with open(default_path + "sumdata/train/valid.article.filter.txt", "r", encoding="utf-8") as f1:
            print("Hello")
            inputs_ = list()
            outputs_ = list()
            for index, inputs in enumerate(f1):
                if((index+1) % args.batch_size == 0 ):
                    #print(inputs_)
                    inputs_.append(inputs)
                    #outputs_.append(outputs)
                    inputss = build_dataset("valid", inputs_, outputs_, word_dict, article_max_len, summary_max_len)
                    print("here it is working better")
                    inputss = np.array(inputss)
                    batch_x_len = list(map(lambda x: len([y for y in x if y != 0]), inputss))
                    valid_feed_dict = {
                        model.batch_size: len(inputss),
                        model.X: inputss,
                        model.X_len: batch_x_len,
                    }
                    prediction = sess.run(model.prediction, feed_dict=valid_feed_dict)
                    prediction_output = [[reversed_dict[y] for y in x] for x in prediction[:, 0, :]]
                    summary_array = []
                    with open(default_path + "result.txt", "w+") as f:
                        for line in prediction_output:
                            summary = list()
                            summari = ''
                            for word in line:
                                if word == "</s>":
                                    break
                                if word not in summary:
                                    summary.append(word)
                                    summari = summari + word + ' '
                            summary_array.append(" ".join(summary))
                            #print(" ".join(summary), file=f)
                            f.write(summari + "\n")
                    inputs_, outputs_ = [],[]
                else:
                    inputs_.append(inputs)
                    #outputs_.append(outputs)
                if index == 2:
                    break

    print('Summaries have been generated')

Loading dictionary...
Loading dictionary...
Loading validation dataset...
Loading article and reference...


W0710 06:32:16.213886 140553886504832 deprecation.py:323] From <ipython-input-11-d18ee4500905>:35: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0710 06:32:16.231102 140553886504832 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/rnn/python/ops/rnn.py:239: bidirectional_dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
W0710 06:32:16.233667 140553886504832 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/rnn.py:464: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Pl

Loading saved model...


W0710 06:32:16.786676 140553886504832 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/rnn.py:244: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0710 06:32:19.040442 140553886504832 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/seq2seq/python/ops/beam_search_decoder.py:985: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0710 06:32:19.832315 140553886504832 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file 

Writing summaries to 'result.txt'...
Hello
here it is working better
here it is working better
here it is working better
Summaries have been generated


In [16]:
print(len(summary_array))
print(summary_array[0])

1
gm sales down # . percent in december


In [17]:
!pip3 install sumeval
!python3 -m spacy download en

     |████████████████████████████████| 51kB 2.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/85/9b/50/fda7087af57d2efde31d4a85cecda98e683ed58eee07fbff2b
  Stored in directory: /root/.cache/pip/wheels/1e/58/be/1915daa86d8a7c9c1134754846c80f481b8d352329f7bd7fa1
Successfully built sumeval sacrebleu
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
from sumeval.metrics.rouge import RougeCalculator
from sumeval.metrics.bleu import BLEUCalculator

def eval_rouges(refrence_summary,model_summary):
    #refrence_summary = "tokyo shares close up #.## percent"
    #model_summary = "tokyo stocks close up # percent to fresh record high"

    rouge = RougeCalculator(stopwords=True, lang="en")

    rouge_1 = rouge.rouge_n(
                summary=model_summary,
                references=refrence_summary,
                n=1)

    rouge_2 = rouge.rouge_n(
                summary=model_summary,
                references=[refrence_summary],
                n=2)
    
    rouge_l = rouge.rouge_l(
                summary=model_summary,
                references=[refrence_summary])
    
    # You need spaCy to calculate ROUGE-BE
    
    rouge_be = rouge.rouge_be(
                summary=model_summary,
                references=[refrence_summary])

    bleu = BLEUCalculator()
    bleu_score = bleu.bleu( summary=model_summary,
                        references=[refrence_summary])
    
    return rouge_1, rouge_2,rouge_l,rouge_be,bleu_score

In [19]:
bleu_arr = []
rouge_1_arr  = []
rouge_2_arr  = []
rouge_L_arr  = []
rouge_be_arr = []

from xml.etree import ElementTree
from xml.dom import minidom
from functools import reduce

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")
  
from xml.etree.ElementTree import Element, SubElement, Comment

top = Element('ZakSum')

comment = Comment('Generated by Prbha Janagama')
top.append(comment)

i=0
for summ in summary_array:
  example = SubElement(top, 'example')
  article_element   = SubElement(example, 'article')
  article_element.text = article[i]
  
  reference_element = SubElement(example, 'reference')
  reference_element.text = reference[i]
  
  summary_element   = SubElement(example, 'summary')
  summary_element.text = summ

  rouge_1, rouge_2,rouge_L,rouge_be,bleu_score = eval_rouges(reference[i],summ )
  print("Rouge 1 : ",rouge_1)
  print("Rouge 2 : ",rouge_2)
  print("Rouge L : ",rouge_L)
  
  eval_element = SubElement(example, 'eval')
  bleu_score_element = SubElement(eval_element,'BLEU', {'score':str(bleu_score)})
  ROUGE_1_element  = SubElement(eval_element, 'ROUGE_1' , {'score':str(rouge_1)})
  ROUGE_2_element  = SubElement(eval_element, 'ROUGE_2' , {'score':str(rouge_2)})
  ROUGE_L_element  = SubElement(eval_element, 'ROUGE_l' , {'score':str(rouge_L)})
  ROUGE_be_element  = SubElement(eval_element,'ROUGE_be', {'score':str(rouge_be)})
  
  bleu_arr.append(bleu_score) 
  rouge_1_arr.append(rouge_1) 
  rouge_2_arr.append(rouge_2) 
  rouge_L_arr.append(rouge_L) 
  rouge_be_arr.append(rouge_be) 

  i+=1
  if(i==1):
    break

top.set('bleu', str(reduce(lambda x, y: x + y,  bleu_arr) / len(bleu_arr)))
top.set('rouge_1', str(reduce(lambda x, y: x + y,  rouge_1_arr) / len(rouge_1_arr)))
top.set('rouge_2', str(reduce(lambda x, y: x + y,  rouge_2_arr) / len(rouge_2_arr)))
top.set('rouge_L', str(reduce(lambda x, y: x + y,  rouge_L_arr) / len(rouge_L_arr)))
top.set('rouge_be', str(reduce(lambda x, y: x + y, rouge_be_arr) / len(rouge_be_arr)))

with open(default_path + "result_valid_29_10_2018_5_28pm.xml", "w+") as f:
  print(prettify(top), file=f)

a.sales=(pobj)=>gm
<BasicElement: sales-[pobj]->gm>
a.percent=(npadvmod)=>gm
<BasicElement: percent-[npadvmod]->gm>
a.injury=(nsubj)=>leaves
<BasicElement: injury-[nsubj]->leave>
a.kwan=(dobj)=>leaves
<BasicElement: kwan-[dobj]->leave>
b.olympic=(amod)=>hopes
Rouge 1 :  0
Rouge 2 :  0
Rouge L :  0


In [20]:
print(summary_array[0])
print(reference[0])
print(article[0])

gm sales down # . percent in december
injury leaves kwan 's olympic hopes in limbo
five-time world champion michelle kwan withdrew from the # us figure skating championships on wednesday , but will petition us skating officials for the chance to compete at the # turin olympics .
